In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data



/home/ayush/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [3]:
X = tf.placeholder(tf.float32, [None, 28 * 28]) # This is for my 28x28 mnist dataset
Z = tf.placeholder(tf.float32, [None, 128]) # Noise

In [4]:
# my generator network:
G_W1 = tf.Variable(tf.random_normal([128, 512], stddev=0.01))
G_W2 = tf.Variable(tf.random_normal([512, 28 * 28], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([512]))
G_b2 = tf.Variable(tf.zeros([28 * 28]))

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
#my generator
def generator(noise_signal): 
    hidden = tf.nn.relu(tf.matmul(noise_signal, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    return output

In [6]:
# my discriminator network
D_W1 = tf.Variable(tf.random_normal([28 * 28, 512], stddev=0.01))
D_W2 = tf.Variable(tf.random_normal([512, 1], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([512]))
D_b2 = tf.Variable(tf.zeros([1]))

In [7]:
#my discriminator
def discriminator(signal): 
    hidden = tf.nn.relu(tf.matmul(signal, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    return output

In [8]:
G = generator(Z)
D_loss = -tf.reduce_mean(tf.log(discriminator(X)) + tf.log(1 - discriminator(G)))
G_loss = -tf.reduce_mean(tf.log(discriminator(G)))
D_train = tf.train.AdamOptimizer(learning_rate=0.0002).minimize(D_loss, var_list=[D_W1, D_b1, D_W2, D_b2])
G_train = tf.train.AdamOptimizer(learning_rate=0.0002).minimize(G_loss, var_list=[G_W1, G_b1, G_W2, G_b2])


In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())


In [ ]:
##################################### training ##########################################
noise_test = np.random.normal(size=(10, 128)) # 10 = Test Sample Size, 128 = Noise Dimension
for epoch in range(200): # Epoch
    for i in range(int(mnist.train.num_examples / 100)): # Batch Size = 100
        batch_xs, _ = mnist.train.next_batch(100)
        noise = np.random.normal(size=(100, 128))

        sess.run(D_train, feed_dict={X: batch_xs, Z: noise})
        sess.run(G_train, feed_dict={Z: noise})

    if epoch == 0 or (epoch + 1) % 10 == 0: # 10 = Saving Period
        samples = sess.run(G, feed_dict={Z: noise_test})

        fig, ax = plt.subplots(1, 10, figsize=(10, 1))
        for i in range(10):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
        plt.savefig('./{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)